In [1]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/dask_xgboost/__init__.py:7: UserWarning: Dask-XGBoost has been deprecated and is no longer maintained. The functionality of this project has been included directly in XGBoost. To use Dask and XGBoost together, please use ``xgboost.dask`` instead https://xgboost.readthedocs.io/en/latest/tutorials/dask.html.
  warnings.warn(


In [2]:
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error

In [11]:
data = pd.read_csv('./Data/ModelData/train_data.csv')
data.drop(columns='Unnamed: 0', axis=1, inplace=True)

y = data.price
X = data.drop(['price'], axis=1)


/var/folders/2j/qtkc2d3d6ld7y6mhr39mt2_w0000gn/T/ipykernel_23021/2043701635.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('./Data/ModelData/train_data.csv')


In [15]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2)

In [16]:
s = (X.dtypes == 'object')
object_cols = list(s[s].index)

In [17]:
from sklearn.metrics import r2_score

numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

categorical_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and 
                        X_train[cname].dtype == "object"]

my_cols = categorical_cols + numerical_cols

X_train_edit = X_train[my_cols].copy()
X_valid_edit = X_valid[my_cols].copy()

numerical_transformer = SimpleImputer(strategy='constant')

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
])

model = RandomForestRegressor(n_estimators=150, random_state=1)

pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])

pipeline.fit(X_train, y_train)

preds = pipeline.predict(X_valid)

r2_score(y_valid, preds)

0.89828579417269

In [18]:
import joblib

filename = 'SUV_model.pk1'
joblib.dump(pipeline, filename, compress=1)

['SUV_model.pk1']

In [19]:
test_load = joblib.load('SUV_model.pk1')

test_data = pd.read_csv('Data/ModelData/test_data.csv')

test_X = test_data.drop(columns='price', axis=1)
test_Y = test_data.price

preds = test_load.predict(test_X)

r2_score(test_Y, preds)

0.8724852981032558